In [3]:
import pandas as pd
import os

In [9]:
#new_car_detail
def load_column_names(df:pd.DataFrame):
    #get car detail
    new_car_detail_column_names =  list(eval(df['new_car_detail'][0]).keys())

    #new_car_overview
    new_car_overview_column_names = list(map['key'] for map in eval(df['new_car_overview'][0])['top'])

    #new car feature
    feature_heading_value_dict = {}
    for idx in range(df.shape[0]):
        item = eval(df.new_car_feature[idx])['data']
        for element in item:
            heading_data = element['heading']
            if heading_data not in feature_heading_value_dict:
                feature_heading_value_dict[heading_data] = set()
            feature_heading_value_dict.get(heading_data).update([val['value'] for val in element['list']])


    new_car_feature_column_names = [col_name for ls in feature_heading_value_dict.values() for col_name in ls]

    #new_car_specs
    specs_heading_value_dict = {}
    for idx in range(df.shape[0]):
        item = eval(df.new_car_specs[idx])['data']
        for element in item:
            heading_data = element['heading']
            if heading_data not in specs_heading_value_dict:
                specs_heading_value_dict[heading_data] = set()
            specs_heading_value_dict.get(heading_data).update([val['key'] for val in element['list']])


    new_car_specs_column_names = [col_name for ls in specs_heading_value_dict.values() for col_name in ls]

    return new_car_detail_column_names,new_car_overview_column_names,new_car_feature_column_names,new_car_specs_column_names




def get_car_detail(df:pd.DataFrame,new_car_detail_column_names,location_info):
    new_car_detail_extracted_values = []
    for idx, val in df['new_car_detail'].items():
        val = eval(val)
        value = []
        if len(new_car_detail_column_names)-1 != len(val.keys()):
            raise Exception("Error")
            break
        for key in new_car_detail_column_names:
            if key == 'location':
                value.append(location_info)
            else:
                value.append(val[key])

        new_car_detail_extracted_values.append(value)

    return pd.DataFrame(new_car_detail_extracted_values,columns= new_car_detail_column_names)


def get_new_car_overview(df:pd.DataFrame, new_car_overview_column_names):
    new_car_overview_extracted_values = []
    for idx in range(rows_length): 
        extracted_information = []
        value = {map['key']:map['value'] for map in eval(df['new_car_overview'][idx])['top']}
        for key in new_car_overview_column_names:
            extracted_information.append(value.get(key,None))
        new_car_overview_extracted_values.append(extracted_information)
    
    return  pd.DataFrame(new_car_overview_extracted_values,columns=new_car_overview_column_names)


def get_new_car_feature(df:pd.DataFrame,new_car_feature_column_names):
    extracted_values = []
    for idx in range(rows_length):
        data = eval(df.new_car_feature[idx])['data']
        data_result = [False]*len(new_car_feature_column_names)
        features = []
        for item in data:
            features.extend([ls_item['value'] for ls_item in item['list']])

        for feature in features:
            feature_index = new_car_feature_column_names.index(feature)
            data_result[feature_index] = True
        
        extracted_values.append(data_result)

    return pd.DataFrame(extracted_values,columns=new_car_feature_column_names)


def get_new_car_specs(df:pd.DataFrame, new_car_specs_column_names):
    #extracting values
    new_specs_extracted_values = []
    for idx in range(rows_length):
        data = eval(df.new_car_specs[idx])['data']
        specs = [None]*len(new_car_specs_column_names)
        for item in data:
            for ls_item in item['list']:
                key = ls_item['key']
                ele_idx = new_car_specs_column_names.index(key)
                specs[ele_idx] = ls_item['value']

        new_specs_extracted_values.append(specs)
    


    return pd.DataFrame(new_specs_extracted_values,columns=new_car_specs_column_names)


    

In [10]:
final_data

NameError: name 'final_data' is not defined

In [11]:
dataset_location = '../dataset/'
column_names = load_column_names(pd.read_excel("../dataset/bangalore_cars.xlsx"))
column_names[0].append('location')
all_column_names = column_names[0]+column_names[1]+column_names[2]+column_names[3]

final_data = pd.DataFrame(columns=all_column_names)

for file in os.listdir(dataset_location):
    location = file.split('_')[0]
    df = pd.read_excel(dataset_location+file)
    rows_length = df.shape[0]
    new_car_detail_extracted_values = get_car_detail(df,column_names[0],location)
    new_car_overview_extracted_values = get_new_car_overview(df,column_names[1])
    new_car_features_extracted_values = get_new_car_feature(df,column_names[2])
    new_specs_extracted_values = get_new_car_specs(df, column_names[3])
    concatenated_data = pd.concat([new_car_detail_extracted_values,new_car_overview_extracted_values,
                                   new_car_features_extracted_values,new_specs_extracted_values],axis=1)
    final_data = pd.concat([final_data,concatenated_data],axis=0)
    





    
    


/tmp/ipykernel_24027/818126414.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat([final_data,concatenated_data],axis=0)


In [ ]:
#older version 2

import glob
import os
import numpy as np
from dotenv import load_dotenv
import pandas as pd
from tqdm import tqdm
import cv2
from ike.models.builder import ModelBuilder
from ike.utils.feature_analysis.geometry_utils import vertical_pole_location
from ike.utils.feature_analysis.image_utils import (create_overlay_image)
from ike.utils.feature_analysis.standard_results_processing import split_into_instances

# from ike.models.instance_seg import build_instance_seg

load_dotenv()
INSIGHT_MODELS_PATH = os.getenv("INSIGHT_MODELS_PATH")
DATA_PATH = os.getenv("DATA_PATH")

mb = ModelBuilder()
pole_model = mb('pole')
wire_model = mb("swi_wire")

image_paths = ["/home/dual10701/Desktop/SVSR/0rGEYA6XSI.jpg"]
for path in image_paths: 
    image = cv2.imread(path)
    image_h,image_w = image.shape[:-1]
    image_name = path.split('/')[-1]
    pole_res = pole_model(image)
    wire_res = wire_model(image)
    pole_bbx = pole_res['bboxes'][0]
    pole_center = int((pole_bbx[0]+pole_bbx[2])/2), int((pole_bbx[1]+pole_bbx[3])/2)

    left_wires_idx = []
    right_wires_idx = []

    left_bboxes = []
    right_bboxes = []

    if not wire_res == None:
            for wire_idx in range(len(wire_res['classes'])):
                wire_bbox = wire_res['bboxes'][wire_idx]
                wire_center = int((wire_bbox[0]+wire_bbox[2])/2), int((wire_bbox[1]+wire_bbox[3])/2)
                if wire_center[1] < (pole_center[1]):
                    if wire_center[0] < pole_center[0]:
                        left_wires_idx.append(wire_idx)
                    else:
                        right_wires_idx.append(wire_idx)


            classified_top_wires = {'classes':[],'masks': np.empty((0,image_h, image_w), dtype=bool)}
            if len(left_wires_idx)>0:
                left_topmost_index = left_wires_idx[np.argmin([wire_res["bboxes"][wire_idx][3] for wire_idx in left_wires_idx])]
                classified_top_wires['classes'].append('wire')
                classified_top_wires['masks'] = np.vstack([classified_top_wires['masks'],np.array([wire_res['masks'][left_topmost_index]])])

            if len(right_wires_idx)>0:
                right_topmost_index = right_wires_idx[np.argmin([wire_res["bboxes"][wire_idx][1] for wire_idx in right_wires_idx])]
                classified_top_wires['classes'].append('transformer')
                classified_top_wires['masks'] = np.vstack([classified_top_wires['masks'],np.array([wire_res['masks'][right_topmost_index]])])

            if len(left_wires_idx)>0 or len(right_wires_idx) >0 : 
                image = create_overlay_image(classified_top_wires,image) 
                cv2.imwrite(f"/home/dual10701/Desktop/{image_name}",image)


            

In [ ]:
#older version -1 of top wire

import glob
import os
import numpy as np
from dotenv import load_dotenv
import pandas as pd
from tqdm import tqdm
import cv2
from ike.models.builder import ModelBuilder
from ike.utils.feature_analysis.geometry_utils import vertical_pole_location
from ike.utils.feature_analysis.image_utils import (create_overlay_image)
from ike.utils.feature_analysis.standard_results_processing import split_into_instances

# from ike.models.instance_seg import build_instance_seg

load_dotenv()
INSIGHT_MODELS_PATH = os.getenv("INSIGHT_MODELS_PATH")
DATA_PATH = os.getenv("DATA_PATH")

mb = ModelBuilder()
pole_model = mb('pole')
wire_model = mb("swi_wire")

image_paths = glob.glob("/home/dual10701/Desktop/SVSR/*.jpg")[32:50]

for path in image_paths: 
    image = cv2.imread(path)
    image_h,image_w = image.shape[:-1]
    image_name = path.split('/')[-1]
    pole_res = pole_model(image)
    wire_res = wire_model(image)
    pole_bbx = pole_res['bboxes'][0]
    pole_center = int((pole_bbx[0]+pole_bbx[2])/2), int((pole_bbx[1]+pole_bbx[3])/2)

    left_wires_idx = []
    right_wires_idx = []

    left_bboxes = []
    right_bboxes = []

    if not wire_res == None:
            for wire_idx in range(len(wire_res['classes'])):
                wire_bbox = wire_res['bboxes'][wire_idx]
                wire_center = int((wire_bbox[0]+wire_bbox[2])/2), int((wire_bbox[1]+wire_bbox[3])/2)
                if wire_center[0] < pole_center[0]:
                    left_wires_idx.append(wire_idx)
                else:
                    right_wires_idx.append(wire_idx)


            classified_top_wires = {'classes':[],'masks': np.empty((0,image_h, image_w), dtype=bool)}
            if len(left_wires_idx)>0:
                left_topmost_index = left_wires_idx[np.argmin([wire_res["bboxes"][wire_idx][3] for wire_idx in left_wires_idx])]
                classified_top_wires['classes'].append('wire')
                classified_top_wires['masks'] = np.vstack([classified_top_wires['masks'],np.array([wire_res['masks'][left_topmost_index]])])

            if len(right_wires_idx)>0:
                right_topmost_index = right_wires_idx[np.argmin([wire_res["bboxes"][wire_idx][1] for wire_idx in right_wires_idx])]
                classified_top_wires['classes'].append('transformer')
                classified_top_wires['masks'] = np.vstack([classified_top_wires['masks'],np.array([wire_res['masks'][right_topmost_index]])])

            if len(left_wires_idx)>0 or len(right_wires_idx) >0 : 
                image = create_overlay_image(classified_top_wires,image)
                cv2.imwrite(f"/home/dual10701/Downloads/test_images-20240711T060233Z-001/outputs/to_learn_pole_ot/top_wire/top_wire_pred/{image_name}",image)


            